In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://stackoverflow.com/jobs/companies"

connection_attempts = 3

while connection_attempts > 0:
    response = requests.get(url)

    if response.status_code == 200:
        print("Successfully connected")
        soup = BeautifulSoup(response.text, "html")
        print(soup)
        break

    elif response.status_code == 429:
         print("Response [429], Reattempting to connect")
         connection_attempts -= 1

    elif response.status_code == 403:
            print("Access denied") 
            break
    else:
         print("Invalid link")
         break




In [ ]:
soup.find_all("a", class_="s-link")

In [4]:
company_list = soup.find_all("a", class_="s-link", href=lambda href: href and "/jobs/companies" in href)
company_list

[<a class="s-link" href="/jobs/companies/intuit?c=OMBF8QvlE61gqGbu">Intuit</a>,
 <a class="s-link" href="/jobs/companies/discover-financial-svcs?c=OKWV0MuOfHpBFieI">Discover Financial Svcs</a>,
 <a class="s-link" href="/jobs/companies/discover-financial-services?c=OJMorUBIUPzuLXGw">Discover Financial Services</a>,
 <a class="s-link" href="/jobs/companies/global-industrial?c=OQpmYaDSuq07PvtC">Global Industrial</a>,
 <a class="s-link" href="/jobs/companies/aldi-s%C3%BCd-it?c=OTIReiEXhddrmhna">ALDI SÜD IT</a>,
 <a class="s-link" href="/jobs/companies/it-sonix-custom-development-gmbh?c=OPeQpiKN9ya0WaVq">IT Sonix Custom Development GmbH</a>,
 <a class="s-link" href="/jobs/companies/jpmorgan-chase-co?c=NjWXyu9b3NnEguru">JPMorgan Chase &amp; Co.</a>,
 <a class="s-link" href="/jobs/companies/adjoe?c=OMpzImRaqhIfhsqs">adjoe </a>,
 <a class="s-link" href="/jobs/companies/embl-ebi-embls-european-bioinformatics-institute?c=OOQFylsqHVxYDJpm">EMBL-EBI (EMBL's European Bioinformatics Institute)</a>,


In [5]:
companies = []

for names in range(len(company_list)):
    companies.append(company_list[names].text.strip())

companies

['Intuit',
 'Discover Financial Svcs',
 'Discover Financial Services',
 'Global Industrial',
 'ALDI SÜD IT',
 'IT Sonix Custom Development GmbH',
 'JPMorgan Chase & Co.',
 'adjoe',
 "EMBL-EBI (EMBL's European Bioinformatics Institute)",
 'Sartorius']

In [6]:
# link to the first company's profile page
# note: link is missing "https://stackoverflow.com"
company_list[0]["href"]

'/jobs/companies/intuit?c=OMBF8QvlE61gqGbu'

In [7]:
# get link to each company page

company_links = []

for names in range(len(company_list)):
    company_links.append("https://stackoverflow.com" + company_list[names]["href"])

company_links


['https://stackoverflow.com/jobs/companies/intuit?c=OMBF8QvlE61gqGbu',
 'https://stackoverflow.com/jobs/companies/discover-financial-svcs?c=OKWV0MuOfHpBFieI',
 'https://stackoverflow.com/jobs/companies/discover-financial-services?c=OJMorUBIUPzuLXGw',
 'https://stackoverflow.com/jobs/companies/global-industrial?c=OQpmYaDSuq07PvtC',
 'https://stackoverflow.com/jobs/companies/aldi-s%C3%BCd-it?c=OTIReiEXhddrmhna',
 'https://stackoverflow.com/jobs/companies/it-sonix-custom-development-gmbh?c=OPeQpiKN9ya0WaVq',
 'https://stackoverflow.com/jobs/companies/jpmorgan-chase-co?c=NjWXyu9b3NnEguru',
 'https://stackoverflow.com/jobs/companies/adjoe?c=OMpzImRaqhIfhsqs',
 'https://stackoverflow.com/jobs/companies/embl-ebi-embls-european-bioinformatics-institute?c=OOQFylsqHVxYDJpm',
 'https://stackoverflow.com/jobs/companies/sartorius?c=ON5SI2CNHCMOiJA4']

In [76]:
# Scraping Multiple Pages

connection_attempts = 3
max_pages = 7

for page in range(1, max_pages + 1):
     url = f"https://stackoverflow.com/jobs/companies?pg={page}"
     
     while connection_attempts > 0:
        response = requests.get(url)

        if response.status_code == 200:
            print("Successfully connected")
            soup = BeautifulSoup(response.text, "html")
            #print("soup")
            break

        elif response.status_code == 429:
            print("Response [429], Reattempting to connect")
            connection_attempts -= 1

        elif response.status_code == 403:
                print("Access denied") 
                break
        else:
            print("Invalid link")
            break

Successfully connected
Successfully connected
Successfully connected
Successfully connected
Successfully connected
Successfully connected
Successfully connected


## Scraping each company profile page

In [77]:
df = pd.DataFrame(
    columns = [["company", "industry", "size", "year_founded", "status", "follower_count", "tech_stack", "benefits"]]
)


In [57]:
# Creating the def function

def scrape_page(soupy):
    company_name = soupy.find("h1").text.strip() # Company Name

    
    about_company = soupy.find_all("span", class_ = "d-block")

    industry = about_company[12].text.strip() # Narrowed search to only industry, size, year founded, status, and followers by calling specific indexes
    size = about_company[13].text.strip() 
    year_founded = about_company[14].text.strip()
    status = about_company[15].text.strip()
    followers = about_company[16].text.strip()


    tech_skills = soupy.find_all("a", class_ = "flex--item s-tag no-tag-menu") # Searched for the company's tech stack
    tech_stack = [] # Created a list containing all relevant tech skills 

    for item in range(len(tech_skills)):
        tech_stack.append(tech_skills[item].text.strip())


    benefits_list = soupy.find_all("div", class_="flex--item pl8 pt2 fw-normal fs-body2 fc-black-700") # Searched for company benefits
    benefits = [] # Created a list containing all listed company benefits

    for item in range(len(benefits_list)):
        benefits.append(benefits_list[item].text.strip())
    
    
    # Appended all webscrapped elements into a list "new_row"

    new_row = [] 

    new_row.append(company_name)
    new_row.append(industry)
    new_row.append(size)
    new_row.append(year_founded)
    new_row.append(status)
    new_row.append(followers)
    new_row.append(tech_stack)
    new_row.append(benefits)

    df.loc[len(df.index)] = new_row # Added the list as a new row in the dataframe


In [63]:
# entering each link 

for link in range(len(company_links)):  # currently testing -> final code would have range(len(company_links))
    response = requests.get(company_links[link])
    max_attempts = 3

    while max_attempts > 0:

        if response.status_code == 200:
            print("Successful Attempt!")
            soupy = BeautifulSoup(response.text, "html")
            scrape_page(soupy)
            break

        elif response.status_code == 429:
            print("Response 429, reattempting...")
            max_attempts -=1

        elif response.status_code == 403:
            print("Access Denied")
            break
        
        else:
            print(response.status_code)
            break

Successful Attempt!
Successful Attempt!
Successful Attempt!
Successful Attempt!
Successful Attempt!
Successful Attempt!
Successful Attempt!
Successful Attempt!
Successful Attempt!
Successful Attempt!


In [78]:
df

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
